In [1]:
import numpy as np
import pandas as pd
import requests
import json
import dotenv
import os

### Getting Biographical Data from the Congress API

The most important skill is reading the API documentation efficiently.

Look ONLY for these five things:

1. Root: the beginning part of all URLs associated with this API for getting data back (I often find this in examples first)

    https://api.congress.gov/v3


2. Endpoint: the second part of the URL, generally it points to a specific dataset to be returned

    /member/{bioguideId}

    The curly braces mean: fill this in for whichever member of congress you want


3. The headers: data that we give to the API to identify outselves, and provide other information if requested
    
    * User Agent string: a string that identifies your software. If you are writing a bot, what is your bot's name, and what methods are you using to access this API.

    The standard convention is: 
    `botname/version (email) python-requests/request-package-version`

In [2]:
botname = 'targ'
version = '0.0'
email = ''
useragent = f'{botname}/{version} ({email}) python-requests/{requests.__version__}'
headers = {'User-Agent': useragent}

4. API Parameters: these are ways to control what data is requested. The parameters are generally listed in the documentation for each endpoint. 3 kinds of parameters:

    * Query parameters: should be listed in a dictionary and passed to the `params` argument of `requests.get()`

    * Path parameters: are part of the endpoint (such as bioguideId)

    * Global parameters: apply to any use of the API regardless of endpoint; often includes your API key; can be in either the headers or the params

5. If an API Key is needed, how to get one, and how will you let the API know the key?

    API keys should be saved in a .env file in the same folder as your code. Use `dotenv` package to load the key into your code without ever displaying/exposing your keys.

In [3]:
dotenv.load_dotenv()
congresskey = os.getenv("congresskey")
params = {'format': 'json',
          'api_key': congresskey}

Put it all together.

In [4]:
bioguideId = 'M001239'
root = 'https://api.congress.gov/v3'
endpoint = f'/member/{bioguideId}'

r = requests.get(root + endpoint, 
                 headers = headers, 
                 params = params)

In [5]:
r.json()

{'member': {'addressInformation': {'city': 'Washington',
   'district': 'DC',
   'officeAddress': '1013 Longworth House Office Building',
   'phoneNumber': '(202) 225-4711',
   'zipCode': 20515},
  'bioguideId': 'M001239',
  'birthYear': '1968',
  'cosponsoredLegislation': {'count': 170,
   'url': 'https://api.congress.gov/v3/member/M001239/cosponsored-legislation'},
  'currentMember': True,
  'depiction': {'attribution': 'Image courtesy of the Member',
   'imageUrl': 'https://www.congress.gov/img/member/67744ba20b34857ecc909149_200.jpg'},
  'directOrderName': 'John J. McGuire III',
  'district': 5,
  'firstName': 'John',
  'honorificName': 'Mr.',
  'invertedOrderName': 'McGuire, John J.',
  'lastName': 'McGuire',
  'middleName': 'J.',
  'officialWebsiteUrl': 'https://mcguire.house.gov/',
  'partyHistory': [{'partyAbbreviation': 'R',
    'partyName': 'Republican',
    'startYear': 2025}],
  'previousNames': [{'directOrderName': 'John McGuire',
    'endDate': '2025-01-10T17:36:18Z',
   

### Sponsored legislation

In [6]:
endpoint = f'/member/{bioguideId}/sponsored-legislation'
endpoint

'/member/M001239/sponsored-legislation'

In [7]:
params = {'format': 'json',
          'offset': 0,
          'limit': 250,
          'api_key': congresskey}

In [8]:
r = requests.get(root + endpoint, 
                 headers=headers, 
                 params=params)
r

<Response [200]>

In [9]:
r.json()

{'pagination': {'count': 9},
 'request': {'bioguideId': 'm001239',
  'contentType': 'application/json',
  'format': 'json'},
 'sponsoredLegislation': [{'congress': 119,
   'introducedDate': '2025-09-26',
   'latestAction': {'actionDate': '2025-09-26',
    'text': 'Referred to the House Committee on Ways and Means.'},
   'number': '5595',
   'policyArea': {'name': None},
   'title': 'Requiring Excise for Migrant Income Transfers Act” or the “REMIT Act.',
   'type': 'HR',
   'url': 'https://api.congress.gov/v3/bill/119/hr/5595?format=json'},
  {'congress': 119,
   'introducedDate': '2025-09-19',
   'latestAction': {'actionDate': '2025-09-19',
    'text': 'Referred to the House Committee on Armed Services.'},
   'number': '5522',
   'policyArea': {'name': None},
   'title': 'Peace Through Strength Act of 2025',
   'type': 'HR',
   'url': 'https://api.congress.gov/v3/bill/119/hr/5522?format=json'},
  {'congress': 119,
   'introducedDate': '2025-09-09',
   'latestAction': {'actionDate': '20

In [10]:
myjson = r.json()
sponsoredLegislation_df = pd.json_normalize(myjson, record_path=['sponsoredLegislation'])

In [11]:
for i in sponsoredLegislation_df['title']:
    print(i)

Requiring Excise for Migrant Income Transfers Act” or the “REMIT Act.
Peace Through Strength Act of 2025
Border Wall Status Act
Make the District of Columbia Safe and Beautiful Act
FAIR Act
VA Data Transparency and Trust Act
Manned Aircraft Clarification Act
Uranium for Energy Independence Act of 2025
Agricultural and Forestry Hauling Efficiency Act


In [12]:
r = requests.get(sponsoredLegislation_df['url'][1], 
                 headers=headers, 
                 params=params)
r.json()

{'bill': {'actions': {'count': 3,
   'url': 'https://api.congress.gov/v3/bill/119/hr/5522/actions?format=json'},
  'committees': {'count': 1,
   'url': 'https://api.congress.gov/v3/bill/119/hr/5522/committees?format=json'},
  'congress': 119,
  'constitutionalAuthorityStatementText': '<pre>\n[Congressional Record Volume 171, Number 154 (Friday, September 19, 2025)]\n[House]\nFrom the Congressional Record Online through the Government Publishing Office [<a href="https://www.gpo.gov">www.gpo.gov</a>]\nBy Mr. McGUIRE:\nH.R. 5522.\nCongress has the power to enact this legislation pursuant\nto the following:\nArticle I, Section 8\n[Page H4489]\n</pre>',
  'introducedDate': '2025-09-19',
  'latestAction': {'actionDate': '2025-09-19',
   'text': 'Referred to the House Committee on Armed Services.'},
  'legislationUrl': 'https://www.congress.gov/bill/119th-congress/house-bill/5522',
  'number': '5522',
  'originChamber': 'House',
  'originChamberCode': 'H',
  'sponsors': [{'bioguideId': 'M0012

### FEC Data for financial campaign contributions

In [13]:
dotenv.load_dotenv()
feckey = os.getenv("feckey")

In [14]:
root = 'https://api.open.fec.gov'
endpoint = '/v1/candidates/search/'

params = {'api_key': feckey,
          'q': 'John McGuire',
          'state': 'VA',
          'district': '5',
          'office': 'H'}

r = requests.get(root + endpoint, 
                 headers=headers, 
                 params=params)
r

<Response [200]>

In [17]:
myjson = r.json()
candidate_id = myjson['results'][0]['candidate_id']
candidate_id

'H0VA07133'

In [ ]:
# three John McGuire

root = 'https://api.open.fec.gov'
endpoint = '/v1/names/candidates/'

params = {'api_key': feckey,
          'q': 'John McGuire'}

r = requests.get(root + endpoint, 
                 headers=headers, 
                 params=params)

r.json()

{'api_version': '1.0',
 'results': [{'id': 'H0VA07133',
   'name': 'MCGUIRE, JOHN',
   'office_sought': 'H'},
  {'id': 'H2VA07196', 'name': 'MCGUIRE, JOHN', 'office_sought': 'H'},
  {'id': 'H0FL16029', 'name': 'MCGUIRE, JOHN MICHAEL', 'office_sought': 'H'}]}

In [ ]:
root = 'https://api.open.fec.gov'
endpoint = f'/v1/candidate/{candidate_id}/committees/'

params = {'api_key': feckey}

r = requests.get(root + endpoint, 
                 headers=headers, 
                 params=params)

[x['name'] for x in r.json()['results']]

['MCGUIRE FOR VIRGINIA', 'MCGUIRE FOR VIRGINIA', 'MCGUIRE RECOUNT COMMITTEE']

In [25]:
committee_ids = [x['committee_id'] for x in r.json()['results']]
committee_ids

['C00727594', 'C00856831', 'C00900118']

In [64]:
root = 'https://api.open.fec.gov'
endpoint = '/v1/schedules/schedule_a/'
params = {'api_key': feckey,
          'committee_id': committee_ids[1],
          'per_page': 100,
          'sort': '-contribution_receipt_amount'}

r = requests.get(root + endpoint, 
                 headers=headers, 
                 params=params)

In [65]:
contrib_dict = [{'contributor_name': x['contributor_name'],
                 'contributor_aggregate_ytd': x['contributor_aggregate_ytd'],
                 'memo_text': x['memo_text']} for x in r.json()['results']]
pd.DataFrame(contrib_dict)

,contributor_name,contributor_aggregate_ytd,memo_text
0,MCGUIRE VICTORY FUND,279791.09,None
1,MCGUIRE VICTORY FUND,119484.10,None
2,MCGUIRE VICTORY FUND,397795.08,None
3,MCGUIRE VICTORY FUND,66488.21,TRANSFER FROM JOINT FUNDRAISER
4,MCGUIRE VICTORY FUND,279791.09,None
...,...,...,...
95,AMERICAN SECURITY PAC,5000.00,None
96,MAJORITY COMMITTEE PAC--MC PAC,10000.00,None
97,MAJORITY COMMITTEE PAC--MC PAC,10000.00,None
98,THE EYE OF THE TIGER PAC,10000.00,None


In [66]:
lastindex = r.json()['pagination']['last_indexes']
lastindex

{'last_contribution_receipt_amount': '5000.00',
 'last_index': '4032120251180734842'}

In [67]:
params['last_contribution_receipt_amount'] = lastindex['last_contribution_receipt_amount']
params['last_index'] = lastindex['last_index']

r = requests.get(root + endpoint, 
                 headers=headers, 
                 params=params)

contrib_dict = [{'contributor_name': x['contributor_name'],
                 'contributor_aggregate_ytd': x['contributor_aggregate_ytd'],
                 'memo_text': x['memo_text']} for x in r.json()['results']]
pd.DataFrame(contrib_dict)

,contributor_name,contributor_aggregate_ytd,memo_text
0,NATIONAL ASSOCIATION OF REALTORS PAC,5000.0,None
1,CHEVRON EMPLOYEES PAC - CHEVRON CORPORATION,5000.0,None
2,AMERICAN ISRAEL PUBLIC AFFAIRS COMMITTEE PAC,5000.0,None
3,SUSAN B ANTHONY LIST INC DBA SUSAN B ANTHONY P...,5000.0,None
4,AMERICAN ISRAEL PUBLIC AFFAIRS COMMITTEE PAC,5000.0,None
...,...,...,...
95,"GROFF, HOWARD",6600.0,None
96,"GROFF, SUSAN",6600.0,None
97,"BOWDEN, MURRY",6600.0,None
98,"CROW, HARLAN",6600.0,None


### Loop over all contributions to McGuire

In [72]:
root = 'https://api.open.fec.gov'
endpoint = '/v1/schedules/schedule_a/'
params = {'api_key': feckey,
          'committee_id': committee_ids[1],
          'per_page': 100,
          'sort': '-contribution_receipt_amount'}

r = requests.get(root + endpoint, 
                 headers=headers, 
                 params=params)

contrib_list = [{'contributor_name': x['contributor_name'],
                 'contributor_aggregate_ytd': x['contributor_aggregate_ytd'],
                 'memo_text': x['memo_text'],
                 'pdf_url': x['pdf_url']} for x in r.json()['results']]

lastindex = r.json()['pagination']['last_indexes']
new = len(r.json()['results'])

while new > 0:
    print(len(contrib_list))

    params['last_contribution_receipt_amount'] = lastindex['last_contribution_receipt_amount']
    params['last_index'] = lastindex['last_index']

    r = requests.get(root + endpoint, headers=headers, params=params)

    contrib_list = contrib_list + [{'contributor_name': x['contributor_name'],
                                    'contributor_aggregate_ytd': x['contributor_aggregate_ytd'],
                                    'memo_text': x['memo_text'],
                                    'pdf_url': x['pdf_url']} for x in r.json()['results']]

    lastindex = r.json()['pagination']['last_indexes']
    new = len(r.json()['results'])

contrib_df = pd.DataFrame(contrib_list)
contrib_df

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1771


,contributor_name,contributor_aggregate_ytd,memo_text,pdf_url
0,MCGUIRE VICTORY FUND,279791.09,None,https://docquery.fec.gov/cgi-bin/fecimg/?20240...
1,MCGUIRE VICTORY FUND,119484.10,None,https://docquery.fec.gov/cgi-bin/fecimg/?20240...
2,MCGUIRE VICTORY FUND,397795.08,None,https://docquery.fec.gov/cgi-bin/fecimg/?20240...
3,MCGUIRE VICTORY FUND,66488.21,TRANSFER FROM JOINT FUNDRAISER,https://docquery.fec.gov/cgi-bin/fecimg/?20250...
4,MCGUIRE VICTORY FUND,279791.09,None,https://docquery.fec.gov/cgi-bin/fecimg/?20240...
...,...,...,...,...
1766,"REHNERT, GEOFFREY",7000.00,REDESIGNATION FROM,https://docquery.fec.gov/cgi-bin/fecimg/?20250...
1767,"GOOD, JOHN P MR",7000.00,REDESIGNATION FROM,https://docquery.fec.gov/cgi-bin/fecimg/?20250...
1768,"PEROT, ROSS",7000.00,MEMO: SPLIT OF-ROSS PEROT,https://docquery.fec.gov/cgi-bin/fecimg/?20250...
1769,"MAFRIGE, DAVID Z.",6500.00,MEMO: SPLIT OF-DAVID Z. MAFRIGE,https://docquery.fec.gov/cgi-bin/fecimg/?20250...
